In [16]:
!pip install pandas
!pip install swifter
!pip install catboost

You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 55.4MB 814kB/s eta 0:00:01    45% |██████████████▌                 | 25.1MB 60.3MB/s eta 0:00:01
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import requests
import json
import subprocess
import os

def download(filename, filename_to_save=None):
  if filename_to_save is None:
    filename_to_save = filename
  url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://yadi.sk/d/pdwdp4Lt5X4DMQ&path=/"+filename
  r = requests.get(url)
  
  download_url = json.loads(r.text)['href']
#   print(download_url)
  subprocess.call(['curl','-L', '-o', filename_to_save, download_url])

  
files_to_download = ["train_part_1_v2.csv.gz", "train_part_2_v2.csv.gz", "test_private_v3_track_1.csv.gz"]

for file in files_to_download:
  if not os.path.exists(file):
    print(file)
    download(file)



train_part_1_v2.csv.gz
train_part_2_v2.csv.gz
test_private_v3_track_1.csv.gz


In [2]:
!git clone https://github.com/yandexdataschool/IDAO-2019-muon-id.git
!cp IDAO-2019-muon-id/* ./

Cloning into 'IDAO-2019-muon-id'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 136 (delta 55), reused 113 (delta 33), pack-reused 0
Receiving objects: 100% (136/136), 1.06 MiB | 2.27 MiB/s, done.
Resolving deltas: 100% (55/55), done.
cp: -r not specified; omitting directory 'IDAO-2019-muon-id/track_2_baseline_advanced_cpp'
cp: -r not specified; omitting directory 'IDAO-2019-muon-id/track_2_baseline_simple_python'


# PROCESS DATA

In [6]:
import pandas as pd
import swifter
import utils
import numpy as np
import os
from itertools import repeat
from tqdm import tqdm   as tqdm

FOR_FEATURE_GEN = ['Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_Y[1]',
       'Lextra_Y[2]', 'Lextra_Y[3]']

def load_foi(path, name):
    converters = dict(zip(utils.FOI_COLUMNS, repeat(utils.parse_array)))
    data = pd.read_csv(os.path.join(path, name),
                       index_col="id", converters=converters,
                       usecols=[utils.ID_COLUMN] + FOR_FEATURE_GEN +  utils.FOI_COLUMNS)
    return data


In [7]:
train = pd.concat([load_foi("./", "train_part_1_v2.csv.gz"), load_foi("./", "train_part_2_v2.csv.gz")])

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
%%time
closest_hits_features_train = train.swifter.progress_bar(True).apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████| 5445705/5445705 [1:27:53<00:00, 1032.68it/s]

CPU times: user 1h 25min 9s, sys: 10min 36s, total: 1h 35min 46s
Wall time: 1h 27min 56s


In [9]:
closest_hits_features_train.to_pickle("gened_features.pckl")

# TRAIN

In [10]:
import numpy as np
import pandas as pd
np.random.seed(42)

In [11]:
drop_features = ['Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_Y[1]',
       'Lextra_Y[2]', 'Lextra_Y[3]', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]']
import utils
features = utils.SIMPLE_FEATURE_COLUMNS + utils.TRAIN_COLUMNS + ["id"]
for i in drop_features:
    features.remove(i)

In [12]:
import pandas as pd
closest_hits_features_train = pd.read_pickle("gened_features.pckl")

In [13]:
train = pd.concat([pd.read_csv("train_part_1_v2.csv.gz", usecols = features, index_col="id"),
                  pd.read_csv("train_part_2_v2.csv.gz", usecols = features, index_col="id")  
])
train = pd.concat([train, closest_hits_features_train], axis=1)

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
import gc
del(closest_hits_features_train)
gc.collect()

28

In [17]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1070, depth=8, learning_rate=0.10, random_seed=42, thread_count= 8)


In [18]:
model.fit(train.drop(['label', 'weight'], axis=1).values, train.label.values, sample_weight=np.maximum(0, train.weight.values))

0:	learn: 0.6572353	total: 1.67s	remaining: 29m 50s
1:	learn: 0.6295515	total: 3.03s	remaining: 26m 58s
2:	learn: 0.6084115	total: 4.48s	remaining: 26m 33s
3:	learn: 0.5918402	total: 5.82s	remaining: 25m 51s
4:	learn: 0.5765303	total: 7.36s	remaining: 26m 7s
5:	learn: 0.5648277	total: 8.58s	remaining: 25m 20s
6:	learn: 0.5545359	total: 10.1s	remaining: 25m 28s
7:	learn: 0.5443710	total: 11.6s	remaining: 25m 39s
8:	learn: 0.5373220	total: 13.1s	remaining: 25m 43s
9:	learn: 0.5306853	total: 14.2s	remaining: 25m 9s
10:	learn: 0.5244087	total: 15.4s	remaining: 24m 43s
11:	learn: 0.5196746	total: 16.8s	remaining: 24m 38s
12:	learn: 0.5150442	total: 18s	remaining: 24m 23s
13:	learn: 0.5100739	total: 19.4s	remaining: 24m 19s
14:	learn: 0.5060260	total: 20.6s	remaining: 24m 12s
15:	learn: 0.5023741	total: 21.9s	remaining: 23m 59s
16:	learn: 0.4986867	total: 23.2s	remaining: 23m 54s
17:	learn: 0.4958682	total: 24.5s	remaining: 23m 50s
18:	learn: 0.4925718	total: 26s	remaining: 23m 56s
19:	learn

154:	learn: 0.4281844	total: 3m 13s	remaining: 19m 3s
155:	learn: 0.4280708	total: 3m 14s	remaining: 19m 2s
156:	learn: 0.4278753	total: 3m 16s	remaining: 19m 1s
157:	learn: 0.4276281	total: 3m 17s	remaining: 18m 59s
158:	learn: 0.4273105	total: 3m 18s	remaining: 18m 59s
159:	learn: 0.4269775	total: 3m 20s	remaining: 18m 58s
160:	learn: 0.4266918	total: 3m 21s	remaining: 18m 58s
161:	learn: 0.4264339	total: 3m 22s	remaining: 18m 56s
162:	learn: 0.4263038	total: 3m 24s	remaining: 18m 55s
163:	learn: 0.4261031	total: 3m 25s	remaining: 18m 53s
164:	learn: 0.4259691	total: 3m 26s	remaining: 18m 52s
165:	learn: 0.4258077	total: 3m 27s	remaining: 18m 50s
166:	learn: 0.4256700	total: 3m 28s	remaining: 18m 48s
167:	learn: 0.4255484	total: 3m 29s	remaining: 18m 46s
168:	learn: 0.4253555	total: 3m 31s	remaining: 18m 46s
169:	learn: 0.4251871	total: 3m 32s	remaining: 18m 44s
170:	learn: 0.4246633	total: 3m 33s	remaining: 18m 43s
171:	learn: 0.4244282	total: 3m 34s	remaining: 18m 41s
172:	learn: 0

304:	learn: 0.4001314	total: 6m 27s	remaining: 16m 10s
305:	learn: 0.4000133	total: 6m 28s	remaining: 16m 9s
306:	learn: 0.3998620	total: 6m 29s	remaining: 16m 7s
307:	learn: 0.3997356	total: 6m 30s	remaining: 16m 6s
308:	learn: 0.3995506	total: 6m 32s	remaining: 16m 5s
309:	learn: 0.3994327	total: 6m 33s	remaining: 16m 5s
310:	learn: 0.3992951	total: 6m 35s	remaining: 16m 4s
311:	learn: 0.3991611	total: 6m 36s	remaining: 16m 2s
312:	learn: 0.3989903	total: 6m 37s	remaining: 16m 1s
313:	learn: 0.3988245	total: 6m 38s	remaining: 16m
314:	learn: 0.3986827	total: 6m 40s	remaining: 15m 58s
315:	learn: 0.3986104	total: 6m 41s	remaining: 15m 57s
316:	learn: 0.3984309	total: 6m 42s	remaining: 15m 56s
317:	learn: 0.3982057	total: 6m 43s	remaining: 15m 55s
318:	learn: 0.3980505	total: 6m 45s	remaining: 15m 53s
319:	learn: 0.3979448	total: 6m 46s	remaining: 15m 52s
320:	learn: 0.3978624	total: 6m 47s	remaining: 15m 50s
321:	learn: 0.3977349	total: 6m 48s	remaining: 15m 49s
322:	learn: 0.3976616	

455:	learn: 0.3809520	total: 9m 43s	remaining: 13m 6s
456:	learn: 0.3808417	total: 9m 45s	remaining: 13m 4s
457:	learn: 0.3807652	total: 9m 46s	remaining: 13m 3s
458:	learn: 0.3806677	total: 9m 47s	remaining: 13m 2s
459:	learn: 0.3805272	total: 9m 48s	remaining: 13m
460:	learn: 0.3804286	total: 9m 50s	remaining: 12m 59s
461:	learn: 0.3802622	total: 9m 51s	remaining: 12m 58s
462:	learn: 0.3801524	total: 9m 52s	remaining: 12m 57s
463:	learn: 0.3799964	total: 9m 54s	remaining: 12m 56s
464:	learn: 0.3798719	total: 9m 55s	remaining: 12m 55s
465:	learn: 0.3797014	total: 9m 57s	remaining: 12m 54s
466:	learn: 0.3795887	total: 9m 58s	remaining: 12m 53s
467:	learn: 0.3794419	total: 10m	remaining: 12m 52s
468:	learn: 0.3793129	total: 10m 1s	remaining: 12m 50s
469:	learn: 0.3792186	total: 10m 2s	remaining: 12m 49s
470:	learn: 0.3790949	total: 10m 4s	remaining: 12m 48s
471:	learn: 0.3789953	total: 10m 5s	remaining: 12m 47s
472:	learn: 0.3789222	total: 10m 7s	remaining: 12m 46s
473:	learn: 0.3787537

603:	learn: 0.3648660	total: 13m 1s	remaining: 10m 2s
604:	learn: 0.3647684	total: 13m 2s	remaining: 10m 1s
605:	learn: 0.3646596	total: 13m 3s	remaining: 9m 59s
606:	learn: 0.3645393	total: 13m 4s	remaining: 9m 58s
607:	learn: 0.3643934	total: 13m 6s	remaining: 9m 57s
608:	learn: 0.3643061	total: 13m 7s	remaining: 9m 55s
609:	learn: 0.3642332	total: 13m 8s	remaining: 9m 54s
610:	learn: 0.3641271	total: 13m 9s	remaining: 9m 53s
611:	learn: 0.3640534	total: 13m 11s	remaining: 9m 52s
612:	learn: 0.3639450	total: 13m 12s	remaining: 9m 50s
613:	learn: 0.3638938	total: 13m 13s	remaining: 9m 49s
614:	learn: 0.3637948	total: 13m 15s	remaining: 9m 48s
615:	learn: 0.3637166	total: 13m 16s	remaining: 9m 46s
616:	learn: 0.3635883	total: 13m 17s	remaining: 9m 45s
617:	learn: 0.3634539	total: 13m 19s	remaining: 9m 44s
618:	learn: 0.3633356	total: 13m 20s	remaining: 9m 43s
619:	learn: 0.3632321	total: 13m 22s	remaining: 9m 42s
620:	learn: 0.3631361	total: 13m 23s	remaining: 9m 40s
621:	learn: 0.3630

754:	learn: 0.3505095	total: 16m 22s	remaining: 6m 49s
755:	learn: 0.3504384	total: 16m 23s	remaining: 6m 48s
756:	learn: 0.3503472	total: 16m 24s	remaining: 6m 47s
757:	learn: 0.3503070	total: 16m 26s	remaining: 6m 45s
758:	learn: 0.3502057	total: 16m 27s	remaining: 6m 44s
759:	learn: 0.3501266	total: 16m 28s	remaining: 6m 43s
760:	learn: 0.3500594	total: 16m 29s	remaining: 6m 41s
761:	learn: 0.3499599	total: 16m 31s	remaining: 6m 40s
762:	learn: 0.3499035	total: 16m 32s	remaining: 6m 39s
763:	learn: 0.3497865	total: 16m 33s	remaining: 6m 38s
764:	learn: 0.3497161	total: 16m 35s	remaining: 6m 36s
765:	learn: 0.3495931	total: 16m 36s	remaining: 6m 35s
766:	learn: 0.3495108	total: 16m 38s	remaining: 6m 34s
767:	learn: 0.3494271	total: 16m 39s	remaining: 6m 33s
768:	learn: 0.3493553	total: 16m 40s	remaining: 6m 31s
769:	learn: 0.3492930	total: 16m 42s	remaining: 6m 30s
770:	learn: 0.3491819	total: 16m 43s	remaining: 6m 29s
771:	learn: 0.3490995	total: 16m 45s	remaining: 6m 28s
772:	learn

905:	learn: 0.3383719	total: 19m 44s	remaining: 3m 34s
906:	learn: 0.3383081	total: 19m 45s	remaining: 3m 33s
907:	learn: 0.3382358	total: 19m 46s	remaining: 3m 31s
908:	learn: 0.3381912	total: 19m 47s	remaining: 3m 30s
909:	learn: 0.3381027	total: 19m 49s	remaining: 3m 29s
910:	learn: 0.3379775	total: 19m 50s	remaining: 3m 27s
911:	learn: 0.3379261	total: 19m 51s	remaining: 3m 26s
912:	learn: 0.3378835	total: 19m 52s	remaining: 3m 25s
913:	learn: 0.3378227	total: 19m 54s	remaining: 3m 23s
914:	learn: 0.3377425	total: 19m 55s	remaining: 3m 22s
915:	learn: 0.3376407	total: 19m 56s	remaining: 3m 21s
916:	learn: 0.3375643	total: 19m 57s	remaining: 3m 19s
917:	learn: 0.3375055	total: 19m 59s	remaining: 3m 18s
918:	learn: 0.3374149	total: 20m	remaining: 3m 17s
919:	learn: 0.3373113	total: 20m 2s	remaining: 3m 15s
920:	learn: 0.3372291	total: 20m 3s	remaining: 3m 14s
921:	learn: 0.3371247	total: 20m 4s	remaining: 3m 13s
922:	learn: 0.3370283	total: 20m 6s	remaining: 3m 12s
923:	learn: 0.3369

1055:	learn: 0.3273708	total: 23m 7s	remaining: 18.4s
1056:	learn: 0.3273068	total: 23m 8s	remaining: 17.1s
1057:	learn: 0.3272361	total: 23m 10s	remaining: 15.8s
1058:	learn: 0.3271549	total: 23m 11s	remaining: 14.5s
1059:	learn: 0.3271074	total: 23m 12s	remaining: 13.1s
1060:	learn: 0.3270358	total: 23m 14s	remaining: 11.8s
1061:	learn: 0.3269638	total: 23m 15s	remaining: 10.5s
1062:	learn: 0.3268808	total: 23m 17s	remaining: 9.2s
1063:	learn: 0.3268255	total: 23m 18s	remaining: 7.88s
1064:	learn: 0.3267595	total: 23m 19s	remaining: 6.57s
1065:	learn: 0.3266984	total: 23m 21s	remaining: 5.26s
1066:	learn: 0.3266511	total: 23m 22s	remaining: 3.94s
1067:	learn: 0.3265710	total: 23m 24s	remaining: 2.63s
1068:	learn: 0.3265210	total: 23m 25s	remaining: 1.31s
1069:	learn: 0.3264577	total: 23m 26s	remaining: 0us


In [19]:
model.save_model("track_2_model.cbm")

In [27]:
!cp baseline.cpp IDAO-2019-muon-id/track_2_baseline_advanced_cpp/baseline.cpp;cp track_2_model.cbm IDAO-2019-muon-id/track_2_baseline_advanced_cpp/track_2_model.cbm; cd IDAO-2019-muon-id/track_2_baseline_advanced_cpp; zip archive * -r; mv archive.zip ../../

  adding: LICENSE (deflated 65%)
  adding: Makefile (deflated 54%)
  adding: baseline.cpp (deflated 53%)
  adding: compute_features.cpp (deflated 47%)
  adding: parser.cpp (deflated 73%)
  adding: parser.h (deflated 51%)
  adding: reference_header.h (deflated 69%)
  adding: ripped_evaluator/ (stored 0%)
  adding: ripped_evaluator/ctr_data_generated.h (deflated 85%)
  adding: ripped_evaluator/evaluator.cpp (deflated 71%)
  adding: ripped_evaluator/include/ (stored 0%)
  adding: ripped_evaluator/include/flatbuffers/ (stored 0%)
  adding: ripped_evaluator/include/flatbuffers/stl_emulation.h (deflated 71%)
  adding: ripped_evaluator/include/flatbuffers/registry.h (deflated 61%)
  adding: ripped_evaluator/include/flatbuffers/minireflect.h (deflated 74%)
  adding: ripped_evaluator/include/flatbuffers/flatc.h (deflated 63%)
  adding: ripped_evaluator/include/flatbuffers/code_generators.h (deflated 62%)
  adding: ripped_evaluator/include/flatbuffers/reflection_generated.h (deflated 89%)
  addi